### Data Base Search file

In [2]:
import yaml
from stringMatcher import levenshtein_ratio_and_distance
from time import time

### Load the database into the memory

In [3]:
data = yaml.load(open("DataSets/finalData.yaml", "rb"))

C:\Users\ASUS\AppData\Local\Temp/ipykernel_12992/1746518049.py:1: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(open("DataSets/finalData.yaml", "rb"))


###### Reading the Global Secondary index in the memory

In [5]:
GIS = yaml.load(open("DataSets/data.yaml", "rb"))

C:\Users\ASUS\AppData\Local\Temp/ipykernel_12992/2625719117.py:1: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  GIS = yaml.load(open("DataSets/data.yaml", "rb"))


#### Writing a function to search games by its ```Name```

In [21]:
def getClosestMatch(String) :
    # Initializing time counter
    tic = time()
    # Finding closest match to the input string fromthe Global Secondary Index
    matchRatio = [levenshtein_ratio_and_distance(String, str(key)) for key, _ in GIS.items()]
    bestMatchRatio = max(matchRatio)
    # Now we have to see that if the best match ratio is less than 0.5 it means that our dataset might not have good match for the search..
    if(bestMatchRatio < 0.5) : 
        return {
            "response code" : 404,
            "search time"   : f"{time()-tic} seconds",
            "message"       : "error",
            "response body" : "no match found"
        }
    # Else if we have a match then...
    bestMatchRatio_index = matchRatio.index(bestMatchRatio)
    # Now we have to find the Game_Id of the game which is best match of the String..
    bestMatch_GameName = list(GIS)[bestMatchRatio_index]
    bestMatch_GameId = GIS[bestMatch_GameName]
    # Now we will retrieve the details of the Game from the main datastore with the help of the Game Id as it will act as Primary Key
    gameInfo = data[bestMatch_GameId]
    # Marking the search completion time..
    toc = time()
    # Adding the Game Name to the response
    gameInfo["Name"] = bestMatch_GameName
    # Constructing the response Model...
    responseModel = {
        "response code" : 200,
        "search time" : f"{toc - tic} seconds",
        "message" : "success",
        "response body" : gameInfo
    }
    return responseModel

### Writing a function to make a request and vizualize the data returned by the search function and "Exception Handling"

In [24]:
def request(gameToSearch = "") : 
    # Name of the game should be a string...
    gameToSearch = str(gameToSearch)
    # Checking whether the name of the game is valid or not...
    if(gameToSearch.replace(" ", "").strip() == "") : 
        return f"Invalid Movie Name"
    # Making a request to fetch the game info from the dataset...
    response = getClosestMatch(String = gameToSearch)
    # Checking the response code...
    if response["response code"] == 404 : 
        return f"ERROR : No records found"
    # If everything is fine, then we print the data...
    data_to_print = f"\
    Name   : {response['response body']['Name']}\n\
    Developer Name : {response['response body']['Developer']}\n\
    Publisher Name : {response['response body']['Publisher']}\n\
    Genre : {response['response body']['Genre']}\n\
    Critic Score : {response['response body']['Critic Score']}\n\
    User Score : {response['response body']['User Score']}\n\
    Rating : {response['response body']['Rating']}\n\
    Available Platform : {response['response body']['Platform']}\n\
    Year Of Release : {response['response body']['Year Of Release']}\n\
    "
    print(data_to_print)

In [32]:
request(gameToSearch = "GrandTheftAuto")

    Name   : Grand Theft Auto
    Developer Name : Digital Eclipse
    Publisher Name : Take-Two Interactive
    Genre : Action
    Critic Score : 68.0
    User Score : 7.1
    Rating : M
    Available Platform : GBA
    Year Of Release : 2004.0
    


#### Thankyou 